# ChemGauss4 docking score predictor based on molecular features
Based on results of a virtual screen against an integrin homology model

Takes input in the form: 

[Polar surface area (ang^2), Hydrogen bond acceptor groups, hydrogen bond donor groups, largest ring size, molecular weight (gmol^-1), number of ring systems, number of rotatable bonds]

In [2]:
# Dependencies:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
print('TF version:', tf.__version__)

TF version: 1.13.1


In [3]:
# Import data from .csv:
data = pd.read_csv(r'ALLI_DOCK_DATA.csv')
score = pd.DataFrame(data, columns = ['CG4:SCORE'])
features = pd.DataFrame(data, columns = ['PSA','HBA', 'HBD', 'Max_ring_size', 'MW', 'N_ring_systems', 'N_rot_bonds'])
N = len(features)

In [4]:
# Separate scores into training and testing sets:
training_score = np.array(score.values[0: int(0.8 * N)], dtype=np.float32)
testing_score = np.array(score.values[int(0.8 * N) + 1: N-1], dtype=np.float32)

# Round scores to positive integers:
for i in range(0,len(training_score)):
    training_score[i] = -1 * int(training_score[i])

for i in range(0,len(testing_score)):
    testing_score[i] = -1 * int(testing_score[i])

In [5]:
# Normalize each feature to 0-1:
PSA = np.array(features.PSA, dtype=np.float32)
N_ring_systems = np.array(features.N_ring_systems, dtype=np.float32)
MW = np.array(features.MW, dtype=np.float32)
Maxring = np.array(features.Max_ring_size, dtype=np.float32)
HBD = np.array(features.HBD, dtype=np.float32)
HBA = np.array(features.HBA, dtype=np.float32)
N_rot_bonds = np.array(features.N_rot_bonds, dtype=np.float32)

for i in range(0,len(PSA)):
    PSA[i] = (PSA[i] - min(PSA))/ max(PSA)
    HBA[i] = (HBA[i] - min(HBA))/ max(HBA)
    HBD[i] = (HBD[i] - min(HBD))/ max(HBD)
    Maxring[i] = (Maxring[i] - min(Maxring))/ max(Maxring)
    MW[i] = (MW[i] - min(MW)) / max(MW)
    N_ring_systems[i] = (N_ring_systems[i] - min(N_ring_systems))/ max(N_ring_systems)
    N_rot_bonds[i] = (N_rot_bonds[i] - min(N_rot_bonds))/ max(N_rot_bonds)    

In [6]:
# Combine normalised features, split into training and testing
norm_features = np.stack((PSA, HBD, HBA, MW, Maxring, N_rot_bonds, N_ring_systems), axis=1)
training_features = np.array(norm_features[0: int(0.8 * N)], dtype=np.float32)
testing_features = np.array(norm_features[int(0.8 * N) + 1: N-1], dtype=np.float32)

In [7]:
# Build model: 
model = keras.Sequential([
    # Two 'fully connected' neural layers, layer two returns score probability:
    keras.layers.Dense(7, activation = tf.nn.relu),
    keras.layers.Dense(12, activation = tf.nn.softmax)
])

In [8]:
# Model compilation :
model.compile(optimizer="adam", # How the model is updated based on the loss function
              loss="sparse_categorical_crossentropy", # Loss function
              metrics=["accuracy"]) # Fraction of images correctly identified

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
# Training :
model.fit(training_features, training_score, epochs = 10)

Epoch 1/10
4985/4985 [==============================] - 0s 82us/sample - loss: 2.3669 - acc: 0.2181
Epoch 2/10
4985/4985 [==============================] - 0s 54us/sample - loss: 1.8167 - acc: 0.4056
Epoch 3/10
4985/4985 [==============================] - 0s 42us/sample - loss: 1.5854 - acc: 0.4056
Epoch 4/10
4985/4985 [==============================] - 0s 42us/sample - loss: 1.5323 - acc: 0.4056
Epoch 5/10
4985/4985 [==============================] - 0s 48us/sample - loss: 1.5128 - acc: 0.4056
Epoch 6/10
4985/4985 [==============================] - 0s 42us/sample - loss: 1.5023 - acc: 0.4056
Epoch 7/10
4985/4985 [==============================] - 0s 42us/sample - loss: 1.4959 - acc: 0.4058
Epoch 8/10
4985/4985 [==============================] - 0s 41us/sample - loss: 1.4906 - acc: 0.4056
Epoch 9/10
4985/4985 [==============================] - 0s 41us/sample - loss: 1.4843 - acc: 0.4056
Epoch 10/10
4985/4985 [==============================] - 0s 41us/sample - loss: 1.4802 - acc: 0.4068

In [10]:
# Evaluation: 
print("Test 1/1")
test_loss, test_acc = model.evaluate(testing_features, testing_score)
print("\nTest accuracy =", test_acc)

Test 1/1
1245/1245 [==============================] - 0s 61us/sample - loss: 1.3011 - acc: 0.5896

Test accuracy = 0.58955824


In [27]:
predictions = model.predict(testing_features)
print(testing_features[1])
print(predictions[1])

[0.88864726 0.5        0.8        0.9604872  0.97222227 0.4267578
 0.8       ]
[2.1630908e-04 3.9266091e-04 3.9803088e-04 1.0891138e-03 1.1973833e-02
 6.9844387e-02 1.9471955e-01 1.8330726e-01 4.5998946e-01 7.5688951e-02
 1.9003857e-03 4.8005325e-04]
